# Predicting Student Dropout in Higher Education

In the evolving landscape of higher education, institutions face increasing pressure to improve student retention and academic success. Dropout rates not only reflect individual struggles but also indicate systemic challenges such as financial hardship, academic mismatch, or lack of institutional support.

This project leverages **data-driven techniques** to uncover the hidden patterns behind student dropout and build predictive models to identify at-risk students early.



## Project Overview

The dataset represents a comprehensive record of students from a higher education institution — including demographics, socioeconomic background, academic performance, and enrollment behavior.

The primary objective is to **predict student dropout** using machine learning, providing actionable insights to educators and administrators for timely interventions.


#### This Notebook has following topics covered 

#### 1. Data Understanding & Preparation

#### 2. Exploratory Analysis & Feature Engineering using dashdb, Microsoft SQL

#### 3. Machine Learning Modeling

### IMPORT DATA

In [1]:
import pandas as pd

df_fact = pd.read_csv('data/fact_table.csv')
df_course = pd.read_csv('data/course_dim.csv')
df_student = pd.read_csv('data/student_dim.csv') 
df_parent = pd.read_csv('data/parents_dim.csv')



In [2]:
# rename columns to avoid ambiguity
df_fact.columns = df_fact.columns.str.strip().str.lower().str.replace(' ', '_').str.replace(r'[^\w]', '', regex=True)
df_course.columns = df_course.columns.str.strip().str.lower().str.replace(' ', '_').str.replace(r'[^\w]', '', regex=True)
df_parent.columns = df_parent.columns.str.strip().str.lower().str.replace(' ', '_').str.replace(r'[^\w]', '', regex=True)
df_student.columns = df_student.columns.str.strip().str.lower().str.replace(' ', '_').str.replace(r'[^\w]', '', regex=True)

In [3]:
# print column names to verify
print(df_fact.columns)
print(df_course.columns)
print(df_parent.columns)
print(df_student.columns)

Index(['student_id', 'course_id', 'age_at_enrollment',
       'curricular_units_1st_sem_credited',
       'curricular_units_1st_sem_enrolled',
       'curricular_units_1st_sem_evaluations',
       'curricular_units_1st_sem_approved', 'curricular_units_1st_sem_grade',
       'curricular_units_1st_sem_without_evaluations',
       'curricular_units_2nd_sem_credited',
       'curricular_units_2nd_sem_enrolled',
       'curricular_units_2nd_sem_evaluations',
       'curricular_units_2nd_sem_approved', 'curricular_units_2nd_sem_grade',
       'curricular_units_2nd_sem_without_evaluations', 'debtor',
       'tuition_fees_up_to_date', 'scholarship_holder', 'target'],
      dtype='object')
Index(['course_id', 'course', 'application_mode', 'application_order',
       'daytimeevening_attendance'],
      dtype='object')
Index(['student_id', 'mothers_qualification', 'fathers_qualification',
       'mothers_occupation', 'fathers_occupation'],
      dtype='object')
Index(['student_id', 'gender', 'int

### DATA CLEANING

### Course dimension clearning

In [5]:
df_course.head()

,course_id,course,application_mode,application_order,daytimeevening_attendance
0,1,1,12,1,1
1,2,1,14,1,1
2,3,1,6,1,1
3,4,1,4,1,1
4,5,1,16,1,1


In [6]:
df_course.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419 entries, 0 to 418
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   course_id                  419 non-null    int64
 1   course                     419 non-null    int64
 2   application_mode           419 non-null    int64
 3   application_order          419 non-null    int64
 4   daytimeevening_attendance  419 non-null    int64
dtypes: int64(5)
memory usage: 16.5 KB


In [14]:
df_course.describe()

,course_id,course,application_mode,application_order,daytimeevening_attendance
count,419.000000,419.000000,419.000000,419.000000,419.000000
mean,210.000000,9.596659,8.155131,2.365155,0.906921
std,121.099133,4.510179,4.874726,1.611213,0.290890
min,1.000000,1.000000,1.000000,0.000000,0.000000
25%,105.500000,6.000000,4.000000,1.000000,1.000000
50%,210.000000,10.000000,8.000000,2.000000,1.000000
75%,314.500000,13.000000,12.000000,3.500000,1.000000
max,419.000000,17.000000,18.000000,9.000000,1.000000


daytimeevening_attendance -> can be category

### Parent dimension clearning

In [8]:
df_parent.head()

,student_id,mothers_qualification,fathers_qualification,mothers_occupation,fathers_occupation
0,1,13,10,6,10
1,2,1,3,4,4
2,3,22,27,10,10
3,4,23,27,6,4
4,5,22,28,10,10


In [7]:
df_parent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4424 entries, 0 to 4423
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   student_id             4424 non-null   int64
 1   mothers_qualification  4424 non-null   int64
 2   fathers_qualification  4424 non-null   int64
 3   mothers_occupation     4424 non-null   int64
 4   fathers_occupation     4424 non-null   int64
dtypes: int64(5)
memory usage: 172.9 KB


In [15]:
df_parent.describe()

,student_id,mothers_qualification,fathers_qualification,mothers_occupation,fathers_occupation
count,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000
mean,2212.500000,12.322107,16.455244,7.317812,7.819168
std,1277.243125,9.026251,11.044800,3.997828,4.856692
min,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1106.750000,2.000000,3.000000,5.000000,5.000000
50%,2212.500000,13.000000,14.000000,6.000000,8.000000
75%,3318.250000,22.000000,27.000000,10.000000,10.000000
max,4424.000000,29.000000,34.000000,32.000000,46.000000


### Student dimension clearning

In [9]:
df_student.head()

,student_id,gender,international,age_at_enrollment,previous_qualification,nacionality,marital_status,educational_special_needs
0,1,1,0,20,1,1,1,0
1,2,1,0,19,1,1,1,0
2,3,1,0,19,1,1,1,0
3,4,0,0,20,1,1,1,0
4,5,0,0,45,1,1,2,0


In [10]:
df_student.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4424 entries, 0 to 4423
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   student_id                 4424 non-null   int64
 1   gender                     4424 non-null   int64
 2   international              4424 non-null   int64
 3   age_at_enrollment          4424 non-null   int64
 4   previous_qualification     4424 non-null   int64
 5   nacionality                4424 non-null   int64
 6   marital_status             4424 non-null   int64
 7   educational_special_needs  4424 non-null   int64
dtypes: int64(8)
memory usage: 276.6 KB


In [13]:
df_student.describe()

,student_id,gender,international,age_at_enrollment,previous_qualification,nacionality,marital_status,educational_special_needs
count,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000
mean,2212.500000,0.351718,0.024864,23.265145,2.531420,1.254521,1.178571,0.011528
std,1277.243125,0.477560,0.155729,7.587816,3.963707,1.748447,0.605747,0.106760
min,1.000000,0.000000,0.000000,17.000000,1.000000,1.000000,1.000000,0.000000
25%,1106.750000,0.000000,0.000000,19.000000,1.000000,1.000000,1.000000,0.000000
50%,2212.500000,0.000000,0.000000,20.000000,1.000000,1.000000,1.000000,0.000000
75%,3318.250000,1.000000,0.000000,25.000000,1.000000,1.000000,1.000000,0.000000
max,4424.000000,1.000000,1.000000,70.000000,17.000000,21.000000,6.000000,1.000000


In [16]:
df_student['previous_qualification'].value_counts()

previous_qualification
1     3717
14     219
12     162
3      126
9       45
15      40
16      36
2       23
6       16
7       11
4        8
13       7
17       6
8        4
11       2
5        1
10       1
Name: count, dtype: int64

In [17]:
df_student['nacionality'].value_counts()

nacionality
1     4314
14      38
12      14
9       13
3       13
10       5
16       3
4        3
18       3
8        2
17       2
15       2
11       2
2        2
19       2
13       1
5        1
21       1
20       1
6        1
7        1
Name: count, dtype: int64

In [18]:
df_student['marital_status'].value_counts()

marital_status
1    3919
2     379
4      91
5      25
6       6
3       4
Name: count, dtype: int64

category
1. gender
2. international
3. educational_special_needs

### Fact Table clearning

In [11]:
df_fact.head()

,student_id,course_id,age_at_enrollment,curricular_units_1st_sem_credited,curricular_units_1st_sem_enrolled,curricular_units_1st_sem_evaluations,curricular_units_1st_sem_approved,curricular_units_1st_sem_grade,curricular_units_1st_sem_without_evaluations,curricular_units_2nd_sem_credited,curricular_units_2nd_sem_enrolled,curricular_units_2nd_sem_evaluations,curricular_units_2nd_sem_approved,curricular_units_2nd_sem_grade,curricular_units_2nd_sem_without_evaluations,debtor,tuition_fees_up_to_date,scholarship_holder,target
0,1,6,20,0,0,0,0,0.000000,0,0,0,0,0,0.000000,0,0,1,0,Dropout
1,2,5,19,0,6,6,6,14.000000,0,0,6,6,6,13.666667,0,0,0,0,Graduate
2,3,66,19,0,6,0,0,0.000000,0,0,6,0,0,0.000000,0,0,0,0,Dropout
3,4,344,20,0,6,8,6,13.428571,0,0,6,10,5,12.400000,0,0,1,0,Graduate
4,5,32,45,0,6,9,5,12.333333,0,0,6,6,6,13.000000,0,0,1,0,Graduate


In [12]:
df_fact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4424 entries, 0 to 4423
Data columns (total 19 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   student_id                                    4424 non-null   int64  
 1   course_id                                     4424 non-null   int64  
 2   age_at_enrollment                             4424 non-null   int64  
 3   curricular_units_1st_sem_credited             4424 non-null   int64  
 4   curricular_units_1st_sem_enrolled             4424 non-null   int64  
 5   curricular_units_1st_sem_evaluations          4424 non-null   int64  
 6   curricular_units_1st_sem_approved             4424 non-null   int64  
 7   curricular_units_1st_sem_grade                4424 non-null   float64
 8   curricular_units_1st_sem_without_evaluations  4424 non-null   int64  
 9   curricular_units_2nd_sem_credited             4424 non-null   i

In [20]:
df_fact.describe()

,student_id,course_id,age_at_enrollment,curricular_units_1st_sem_credited,curricular_units_1st_sem_enrolled,curricular_units_1st_sem_evaluations,curricular_units_1st_sem_approved,curricular_units_1st_sem_grade,curricular_units_1st_sem_without_evaluations,curricular_units_2nd_sem_credited,curricular_units_2nd_sem_enrolled,curricular_units_2nd_sem_evaluations,curricular_units_2nd_sem_approved,curricular_units_2nd_sem_grade,curricular_units_2nd_sem_without_evaluations,debtor,tuition_fees_up_to_date,scholarship_holder
count,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000
mean,2212.500000,210.309675,23.265145,0.709991,6.270570,8.299051,4.706600,10.640822,0.137658,0.541817,6.232143,8.063291,4.435805,10.230206,0.150316,0.113698,0.880651,0.248418
std,1277.243125,116.416470,7.587816,2.360507,2.480178,4.179106,3.094238,4.843663,0.690880,1.918546,2.195951,3.947951,3.014764,5.210808,0.753774,0.317480,0.324235,0.432144
min,1.000000,1.000000,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1106.750000,98.000000,19.000000,0.000000,5.000000,6.000000,3.000000,11.000000,0.000000,0.000000,5.000000,6.000000,2.000000,10.750000,0.000000,0.000000,1.000000,0.000000
50%,2212.500000,213.000000,20.000000,0.000000,6.000000,8.000000,5.000000,12.285714,0.000000,0.000000,6.000000,8.000000,5.000000,12.200000,0.000000,0.000000,1.000000,0.000000
75%,3318.250000,299.000000,25.000000,0.000000,7.000000,10.000000,6.000000,13.400000,0.000000,0.000000,7.000000,10.000000,6.000000,13.333333,0.000000,0.000000,1.000000,0.000000
max,4424.000000,419.000000,70.000000,20.000000,26.000000,45.000000,26.000000,18.875000,12.000000,19.000000,23.000000,33.000000,20.000000,18.571429,12.000000,1.000000,1.000000,1.000000


In [21]:
df_fact['debtor'].value_counts()

debtor
0    3921
1     503
Name: count, dtype: int64

In [22]:
df_fact['tuition_fees_up_to_date'].value_counts()

tuition_fees_up_to_date
1    3896
0     528
Name: count, dtype: int64

In [23]:
df_fact['scholarship_holder'].value_counts()

scholarship_holder
0    3325
1    1099
Name: count, dtype: int64

In [25]:
df_fact['target'].value_counts()

target
Graduate    2209
Dropout     1421
Enrolled     794
Name: count, dtype: int64

category-
1. debtor
2. tuition_fees_up_to_date
3. scholarship_holder

### PULL DATA INTO SQL

In [24]:
import duckdb
import pandas as pd

# Connect to database (will create if it doesn't exist)
con = duckdb.connect('retention.db')


# Load data into DuckDB
con.execute("CREATE TABLE IF NOT EXISTS fact_table AS SELECT * FROM df_fact")
con.execute("CREATE TABLE IF NOT EXISTS course_dim AS SELECT * FROM df_course")
con.execute("CREATE TABLE IF NOT EXISTS parents_dim AS SELECT * FROM df_parent")
con.execute("CREATE TABLE IF NOT EXISTS student_dim AS SELECT * FROM df_student")

# Verify tables
tables = con.execute("SHOW TABLES").fetchall()
print(tables)


[('course_dim',), ('fact_table',), ('parents_dim',), ('student_dim',)]


### ANALYSIS


Student Retention Overview


Description:
This query provides a summary of overall student outcomes, identifying the distribution between graduates and dropouts to establish a baseline retention metric.

In [12]:
query1 = """
SELECT 
    f.target AS outcome_status,
    COUNT(*) AS total_students,
    ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM fact_table), 2) AS percentage
FROM fact_table f
GROUP BY f.target
ORDER BY total_students DESC;
"""

# Run a query
result = con.execute(query1).fetchall()
print(result)


[('Graduate', 2209, 49.93), ('Dropout', 1421, 32.12), ('Enrolled', 794, 17.95)]


Retention by Course and Attendance Mode

Description:
Analyzes how retention varies by course and attendance mode (daytime or evening).

In [15]:
query = """
SELECT 
    c.course,
    c.daytimeevening_attendance,
    f.target,
    COUNT(*) AS num_students,
    ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (PARTITION BY c.course), 2) AS pct_within_course
FROM fact_table f
JOIN course_dim c ON f.course_id = c.course_id
GROUP BY c.course, c.daytimeevening_attendance, f.target
ORDER BY c.course, pct_within_course DESC;
"""

# Run a query
result = con.execute(query).fetchall()
print(result)

[(1, 1, 'Dropout', 11, 61.11), (1, 1, 'Graduate', 4, 22.22), (1, 1, 'Enrolled', 3, 16.67), (2, 1, 'Graduate', 96, 44.65), (2, 1, 'Dropout', 82, 38.14), (2, 1, 'Enrolled', 37, 17.21), (3, 0, 'Graduate', 123, 57.21), (3, 0, 'Dropout', 71, 33.02), (3, 0, 'Enrolled', 21, 9.77), (4, 1, 'Graduate', 87, 41.43), (4, 1, 'Dropout', 86, 40.95), (4, 1, 'Enrolled', 37, 17.62), (5, 1, 'Graduate', 133, 58.85), (5, 1, 'Dropout', 51, 22.57), (5, 1, 'Enrolled', 42, 18.58), (6, 1, 'Graduate', 172, 51.04), (6, 1, 'Dropout', 90, 26.71), (6, 1, 'Enrolled', 75, 22.26), (7, 1, 'Dropout', 92, 54.12), (7, 1, 'Enrolled', 64, 37.65), (7, 1, 'Graduate', 14, 8.24), (8, 1, 'Dropout', 78, 55.32), (8, 1, 'Graduate', 42, 29.79), (8, 1, 'Enrolled', 21, 14.89), (9, 1, 'Graduate', 138, 36.32), (9, 1, 'Dropout', 134, 35.26), (9, 1, 'Enrolled', 108, 28.42), (10, 1, 'Graduate', 248, 69.86), (10, 1, 'Dropout', 65, 18.31), (10, 1, 'Enrolled', 42, 11.83), (11, 1, 'Graduate', 112, 45.53), (11, 1, 'Dropout', 93, 37.8), (11, 1, 'E

Gender-Based Academic Outcomes

Description:
Explores gender differences in student outcomes, providing insights into diversity and inclusion challenges

In [16]:
query = """
SELECT 
    s.gender,
    f.target,
    COUNT(*) AS total_students,
    ROUND(AVG(f.curricular_units_1st_sem_grade), 2) AS avg_first_sem_grade,
    ROUND(AVG(f.curricular_units_2nd_sem_grade), 2) AS avg_second_sem_grade
FROM fact_table f
JOIN student_dim s ON f.student_id = s.student_id
GROUP BY s.gender, f.target
ORDER BY s.gender, f.target;
"""

# Run a query
result = con.execute(query).fetchall()
print(result)

[(0, 'Dropout', 720, 7.75, 6.31), (0, 'Enrolled', 487, 11.43, 11.42), (0, 'Graduate', 1661, 12.83, 12.9), (1, 'Dropout', 701, 6.75, 5.48), (1, 'Enrolled', 307, 10.64, 10.64), (1, 'Graduate', 548, 12.08, 12.07)]


Age and Retention Relationship

Description:
Analyzes how age at enrollment affects student performance and graduation likelihood.

In [ ]:
query = """
SELECT 
    s.age_at_enrollment,
    AVG(f.curricular_units_1st_sem_grade) AS avg_first_sem_grade,
    AVG(f.curricular_units_2nd_sem_grade) AS avg_second_sem_grade,
    SUM(CASE WHEN f.target = 'Graduate' THEN 1 ELSE 0 END) AS graduates,
    COUNT(*) AS total_students,
    ROUND(100.0 * SUM(CASE WHEN f.Target = 'Graduate' THEN 1 ELSE 0 END) / COUNT(*), 2) AS retention_rate
FROM fact_table f
JOIN student_dim s ON f.student_id = s.student_id
GROUP BY s.age_at_enrollment
ORDER BY s.age_at_enrollment;
"""

# Run a query
result = con.execute(query).fetchall()
print(result)

[(17, 10.75025, 10.360964286, 3, 5, 60.0), (18, 11.562686583426634, 11.29862881192084, 662, 1036, 63.9), (19, 11.154266521635572, 10.847340578869387, 547, 911, 60.04), (20, 11.190518613739568, 10.864023728397331, 326, 599, 54.42), (21, 10.94130664319876, 10.698387562484477, 159, 322, 49.38), (22, 10.095293217528738, 9.941642663793106, 79, 174, 45.4), (23, 10.588121526388889, 10.122312273518522, 43, 108, 39.81), (24, 10.006360162595419, 9.634906613664123, 45, 131, 34.35), (25, 9.080935325268818, 8.845415056881722, 25, 93, 26.88), (26, 8.887535461170208, 8.000944919680851, 23, 94, 24.47), (27, 8.713101037472528, 7.795334584505496, 24, 91, 26.37), (28, 9.560847720602407, 8.322594375662652, 21, 83, 25.3), (29, 9.189076893939395, 7.7111768283333335, 16, 66, 24.24), (30, 9.04316582489796, 8.885471648367348, 13, 49, 26.53), (31, 9.42272754272727, 8.844194245272726, 12, 55, 21.82), (32, 8.765335980983608, 7.695434731147541, 23, 61, 37.7), (33, 10.503835978666668, 8.948969376222221, 17, 45, 37.

Financial Responsibility and Retention

Description:
Examines whether students with tuition debt or unpaid fees are at higher risk of dropping out.

In [18]:
query = """
SELECT 
    f.debtor,
    f.tuition_fees_up_to_date,
    f.target,
    COUNT(*) AS num_students
FROM fact_table f
GROUP BY f.debtor, f.tuition_fees_up_to_date, f.target
ORDER BY f.debtor DESC, f.tuition_fees_up_to_date DESC;
"""

# Run a query
result = con.execute(query).fetchall()
print(result)

[(1, 1, 'Graduate', 90), (1, 1, 'Dropout', 97), (1, 1, 'Enrolled', 70), (1, 0, 'Enrolled', 20), (1, 0, 'Dropout', 215), (1, 0, 'Graduate', 11), (0, 1, 'Enrolled', 682), (0, 1, 'Dropout', 867), (0, 1, 'Graduate', 2090), (0, 0, 'Enrolled', 22), (0, 0, 'Graduate', 18), (0, 0, 'Dropout', 242)]


Scholarship Impact on Retention

Description:
Evaluates whether scholarship holders demonstrate improved academic outcomes or retention rates.

In [19]:
query = """
SELECT 
    f.scholarship_holder,
    f.target,
    COUNT(*) AS num_students,
    ROUND(AVG(f.curricular_units_1st_sem_grade), 2) AS avg_first_sem_grade,
    ROUND(AVG(f.curricular_units_2nd_sem_grade), 2) AS avg_second_sem_grade
FROM fact_table f
GROUP BY f.scholarship_holder, f.target
ORDER BY f.scholarship_holder, f.target;
"""

# Run a query
result = con.execute(query).fetchall()
print(result)

[(0, 'Dropout', 1287, 7.06, 5.76), (0, 'Enrolled', 664, 11.12, 11.1), (0, 'Graduate', 1374, 12.63, 12.68), (1, 'Dropout', 134, 9.1, 7.21), (1, 'Enrolled', 130, 11.16, 11.19), (1, 'Graduate', 835, 12.67, 12.73)]


Parental Education and Academic Success

Description:
Analyzes how parents’ educational qualifications correlate with student outcomes.

In [20]:
query = """
SELECT 
    p.mothers_qualification,
    p.fathers_qualification,
    f.target,
    COUNT(*) AS num_students,
    ROUND(AVG(f.curricular_units_1st_sem_grade), 2) AS avg_first_sem_grade
FROM fact_table f
JOIN parents_dim p ON f.student_id = p.student_id
GROUP BY p.mothers_qualification, p.fathers_qualification, f.target
ORDER BY num_students DESC;
"""

# Run a query
result = con.execute(query).fetchall()
print(result)

[(22, 27, 'Graduate', 305, 12.65), (22, 27, 'Dropout', 283, 6.55), (1, 1, 'Graduate', 212, 12.71), (13, 14, 'Graduate', 188, 12.64), (1, 14, 'Graduate', 161, 12.9), (1, 1, 'Dropout', 132, 7.92), (23, 28, 'Graduate', 129, 12.86), (22, 27, 'Enrolled', 117, 11.42), (13, 28, 'Graduate', 114, 12.64), (23, 27, 'Graduate', 103, 12.97), (13, 14, 'Dropout', 102, 7.18), (13, 27, 'Graduate', 97, 12.36), (13, 1, 'Graduate', 81, 12.4), (1, 1, 'Enrolled', 80, 11.13), (1, 28, 'Graduate', 76, 12.53), (19, 24, 'Dropout', 74, 6.14), (22, 28, 'Graduate', 71, 13.0), (1, 14, 'Dropout', 67, 7.98), (3, 3, 'Graduate', 65, 12.69), (3, 1, 'Graduate', 64, 12.2), (23, 14, 'Graduate', 62, 12.07), (13, 27, 'Dropout', 60, 7.65), (13, 14, 'Enrolled', 58, 11.08), (1, 27, 'Graduate', 57, 13.03), (13, 1, 'Dropout', 55, 6.26), (23, 27, 'Dropout', 47, 7.31), (1, 14, 'Enrolled', 47, 11.48), (22, 14, 'Graduate', 46, 12.57), (23, 28, 'Dropout', 45, 8.1), (3, 1, 'Dropout', 43, 6.47), (3, 14, 'Graduate', 43, 13.51), (13, 27, '

Course Load and Student Outcomes

Description:
Explores how the number of enrolled curricular units relates to performance and retention.

In [21]:
query = """
SELECT 
    (f.curricular_units_1st_sem_enrolled + f.curricular_units_2nd_sem_enrolled) AS total_enrolled_units,
    AVG(f.curricular_units_1st_sem_grade) AS avg_grade_1st,
    AVG(f.curricular_units_2nd_sem_grade) AS avg_grade_2nd,
    SUM(CASE WHEN f.target = 'Graduate' THEN 1 ELSE 0 END) AS graduates,
    ROUND(100.0 * SUM(CASE WHEN f.target = 'Graduate' THEN 1 ELSE 0 END) / COUNT(*), 2) AS retention_rate
FROM fact_table f
GROUP BY total_enrolled_units
ORDER BY total_enrolled_units;
"""

# Run a query
result = con.execute(query).fetchall()
print(result)

[(0, 0.0, 0.0, 75, 41.67), (3, 10.5, 11.0, 0, 0.0), (4, 0.0, 0.0, 0, 0.0), (5, 10.0520833325, 5.875, 0, 0.0), (6, 5.381666667, 5.883333333, 1, 10.0), (7, 2.0833333333333335, 2.0833333333333335, 0, 0.0), (8, 1.2, 1.283333333, 1, 10.0), (9, 2.875, 1.7583333335, 0, 0.0), (10, 9.638138440836691, 9.380859174909272, 328, 33.06), (11, 5.4793333332, 3.4600238093999995, 2, 4.0), (12, 11.385112896843525, 10.702623540659156, 989, 53.0), (13, 11.304330357187501, 10.095206304375, 6, 18.75), (14, 11.71978369133334, 11.06135527576471, 103, 40.39), (15, 12.015904772559356, 12.181387511372032, 267, 70.45), (16, 12.979847132779556, 12.94748785415336, 254, 81.15), (17, 12.31161844, 12.221595655789473, 7, 36.84), (18, 12.670554282941177, 12.242311282941175, 6, 35.29), (19, 12.434395194347825, 12.644940808260868, 9, 39.13), (20, 12.63896125868421, 12.53824713236842, 24, 63.16), (21, 12.695239482857144, 12.491074005714287, 6, 42.86), (22, 12.725037707058824, 12.621799932941174, 12, 70.59), (23, 13.240839715

International vs Domestic Student Retention

Description:
Compares performance and outcomes between domestic and international students.

In [22]:
query = """
SELECT 
    s.international,
    f.target,
    COUNT(*) AS num_students,
    ROUND(AVG(f.curricular_units_1st_sem_grade), 2) AS avg_first_sem_grade,
    ROUND(AVG(f.curricular_units_2nd_sem_grade), 2) AS avg_second_sem_grade
FROM fact_table f
JOIN student_dim s ON f.student_id = s.student_id
GROUP BY s.international, f.target
ORDER BY s.international, f.target;
"""

# Run a query
result = con.execute(query).fetchall()
print(result)

[(0, 'Dropout', 1389, 7.26, 5.91), (0, 'Enrolled', 770, 11.11, 11.1), (0, 'Graduate', 2155, 12.64, 12.7), (1, 'Dropout', 32, 7.24, 5.28), (1, 'Enrolled', 24, 11.66, 11.53), (1, 'Graduate', 54, 12.6, 12.68)]


In [31]:
df['Target'].value_counts()

Target
Graduate    2209
Dropout     1421
Enrolled     794
Name: count, dtype: int64

### MODEL

In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import pickle

# Load dataset
df = pd.read_csv('data/dataset.csv')

# ---------------- Data Cleaning ----------------
# rename columns to avoid ambiguity
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace(r'[^\w]', '', regex=True)
# Remove rows with excessive missing values
df.dropna(thresh=int(0.5 * df.shape[1]), inplace=True)

# Separate numeric and categorical columns
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
cat_cols = df.select_dtypes(include=['object']).columns

# Impute missing values: mean for numeric, mode for categorical
df[num_cols] = SimpleImputer(strategy='mean').fit_transform(df[num_cols])
df[cat_cols] = SimpleImputer(strategy='most_frequent').fit_transform(df[cat_cols])

# ---------------- Target Preparation ----------------
# Split the dataset:
#   - Graduates (0) and Dropouts (1) → used for training
#   - Enrolled → reserved for later predictions
enrolled_df = df[df['target'] == 'Enrolled'].copy()
train_df = df[df['target'].isin(['Graduate', 'Dropout'])].copy()

# Encode target variable
train_df['target'] = train_df['target'].map({'Graduate': 0, 'Dropout': 1}).astype(int)

# ---------------- Feature Encoding & Scaling ----------------
# Identify columns for scaling and encoding
num_cols = train_df.select_dtypes(include=['int64', 'float64']).columns.drop('target', errors='ignore')
cat_cols = train_df.select_dtypes(include=['object']).columns

# Encode categorical features using LabelEncoder
le = LabelEncoder()
for col in cat_cols:
    train_df[col] = le.fit_transform(train_df[col])
    if col in enrolled_df.columns:
        enrolled_df[col] = le.transform(enrolled_df[col])

# Scale numeric features using StandardScaler
scaler = StandardScaler()
train_df[num_cols] = scaler.fit_transform(train_df[num_cols])

# ---------------- Model Training ----------------
# Split into training and validation sets
X = train_df.drop('target', axis=1)
y = train_df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define multiple candidate models
models = {
    'RandomForest': RandomForestClassifier(),
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'SVM': SVC()
}

# Train each model and collect performance metrics
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results[name] = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, average='weighted'),
        'f1': f1_score(y_test, y_pred, average='weighted')
    }

# ---------------- Model Selection ----------------
# Choose the model with the best F1 score
best_model_name = max(results, key=lambda x: results[x]['f1'])
best_model = models[best_model_name]

print(f"Best model: {best_model_name}")
print("Performance metrics:", results[best_model_name])

# ---------------- Model Saving ----------------
# Save the trained best model to disk
with open('model/best_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)


# Save the scaler for use during prediction
with open('model/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print(" Best model saved successfully to 'model/best_model.pkl'")


Best model: LogisticRegression
Performance metrics: {'accuracy': 0.9201101928374655, 'precision': 0.9207543940014337, 'f1': 0.9193411475853628}
 Best model saved successfully to 'model/best_model.pkl'


### Prediction On New Data

In [ ]:
# # The enrolled group can now be used for prediction
# enrolled_features = enrolled_df.drop('target', axis=1, errors='ignore')
# enrolled_predictions = best_model.predict(enrolled_features)

# # Save predictions
# enrolled_df['predicted_dropout_risk'] = enrolled_predictions
# enrolled_df.to_csv('data/enrolled_predictions.csv', index=False)

# print("Predictions for enrolled students saved to 'data/enrolled_predictions.csv'")